In [1]:
import pandas as pd
import numpy as np
from datasist.structdata import detect_outliers
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
import warnings
warnings.simplefilter(action='ignore')
import joblib

In [2]:
df = pd.read_excel('superstore_sales.xlsx',na_values='NAN')
df

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,EG-2011-1800,2011-01-01,2011-01-05,Standard Class,Hunter Glantz,Consumer,Al 11Iskandariyah,Egypt,Africa,Africa,OFF-BRE-10001343,Office Supplies,Appliances,"Breville Blender, Silver",NaN,6.0,0.0,91.980,92.94,NaN
1,IN-2011-47883,2011-01-01,2011-01-08,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3.0,0.1,36.036,9.72,NaN
2,HU-2011-1220,2011-01-01,2011-01-05,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4.0,0.0,29.640,8.17,NaN
3,IT-2011-3647632,2011-01-01,2011-01-05,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3.0,0.5,-26.055,4.82,NaN
4,IN-2011-47883,2011-01-01,2011-01-08,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.670,5.0,NaN,37.770,4.70,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51480,IS-2014-30,2014-11-25,2014-11-27,Second Class,Sean Christensen,Consumer,Tel Aviv,NaN,EMEA,EMEA,FUR-RUB-10001094,Furniture,Furnishings,"Rubbermaid Door Stop, Duo Pack",43.860,1.0,0.0,3.060,NaN,Critical
51481,IS-2014-3300,2014-12-06,2014-12-09,First Class,Neil Ducich,Corporate,Tel Aviv,NaN,EMEA,EMEA,OFF-TEN-10004270,Office Supplies,Storage,"Tenex File Cart, Single Width",134.160,1.0,0.0,12.060,18.45,Medium
51482,IS-2014-4250,2014-12-17,2014-12-20,First Class,Meg O'Connel,Home Office,Haifa,NaN,EMEA,EMEA,OFF-CUI-10003993,Office Supplies,Appliances,"Cuisinart Microwave, Red",278.460,1.0,0.0,75.180,63.45,High
51483,IS-2014-4250,2014-12-17,2014-12-20,First Class,Meg O'Connel,Home Office,Haifa,NaN,EMEA,EMEA,FUR-HON-10000722,Furniture,Chairs,"Hon Rocking Chair, Adjustable",133.110,1.0,0.0,63.870,31.04,High


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51485 entries, 0 to 51484
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        51485 non-null  object        
 1   order_date      51485 non-null  datetime64[ns]
 2   ship_date       51485 non-null  datetime64[ns]
 3   ship_mode       51134 non-null  object        
 4   customer_name   51066 non-null  object        
 5   segment         51318 non-null  object        
 6   state           51032 non-null  object        
 7   country         51086 non-null  object        
 8   market          51020 non-null  object        
 9   region          51375 non-null  object        
 10  product_id      51458 non-null  object        
 11  category        50867 non-null  object        
 12  sub_category    51481 non-null  object        
 13  product_name    51470 non-null  object        
 14  sales           51400 non-null  float64       
 15  qu

In [4]:
df.drop(['order_id','product_id','customer_name'],axis=1,inplace=True)

In [5]:
cat_col=df.select_dtypes(include='object').columns
for cat in cat_col:
    print('value_counts is :','\n',df[cat].value_counts())
    print("\n",'-'*50)

value_counts is : 
 Standard Class    30664
Second Class      10276
First Class        7494
Same Day           2700
Name: ship_mode, dtype: int64

 --------------------------------------------------
value_counts is : 
 Consumer       26520
Corporate      15442
Home Office     9356
Name: segment, dtype: int64

 --------------------------------------------------
value_counts is : 
 California          1984
England             1496
New York            1128
Ile-de-France        978
Texas                977
                    ... 
Medea                  1
Stara Zagora           1
Arad                   1
Galway                 1
Atsimo-Andrefana       1
Name: state, Length: 1093, dtype: int64

 --------------------------------------------------
value_counts is : 
 United States    9939
Australia        2833
France           2809
Mexico           2637
Germany          2055
                 ... 
South Sudan         2
Burundi             2
Chad                2
Eritrea             2
Bahrain  

### Check for duplicates 

In [6]:
df.duplicated().sum()

38

### Remove Duplicates

In [7]:
df.drop_duplicates(inplace=True)

### Reset Index

In [8]:
df.reset_index(drop=True,inplace=True)

### Apply univariate analysis to check for errors

In [9]:
df

,order_date,ship_date,ship_mode,segment,state,country,market,region,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority
0,2011-01-01,2011-01-05,Standard Class,Consumer,Al 11Iskandariyah,Egypt,Africa,Africa,Office Supplies,Appliances,"Breville Blender, Silver",NaN,6.0,0.0,91.980,92.94,NaN
1,2011-01-01,2011-01-08,Standard Class,Consumer,New South Wales,Australia,APAC,Oceania,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3.0,0.1,36.036,9.72,NaN
2,2011-01-01,2011-01-05,Second Class,Consumer,Budapest,Hungary,EMEA,EMEA,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4.0,0.0,29.640,8.17,NaN
3,2011-01-01,2011-01-05,Second Class,Home Office,Stockholm,Sweden,EU,North,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3.0,0.5,-26.055,4.82,NaN
4,2011-01-01,2011-01-08,Standard Class,Consumer,New South Wales,Australia,APAC,Oceania,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.670,5.0,NaN,37.770,4.70,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51442,2014-11-25,2014-11-27,Second Class,Consumer,Tel Aviv,NaN,EMEA,EMEA,Furniture,Furnishings,"Rubbermaid Door Stop, Duo Pack",43.860,1.0,0.0,3.060,NaN,Critical
51443,2014-12-06,2014-12-09,First Class,Corporate,Tel Aviv,NaN,EMEA,EMEA,Office Supplies,Storage,"Tenex File Cart, Single Width",134.160,1.0,0.0,12.060,18.45,Medium
51444,2014-12-17,2014-12-20,First Class,Home Office,Haifa,NaN,EMEA,EMEA,Office Supplies,Appliances,"Cuisinart Microwave, Red",278.460,1.0,0.0,75.180,63.45,High
51445,2014-12-17,2014-12-20,First Class,Home Office,Haifa,NaN,EMEA,EMEA,Furniture,Chairs,"Hon Rocking Chair, Adjustable",133.110,1.0,0.0,63.870,31.04,High


In [ ]:
num_columns = df.select_dtypes(include="number").columns
for col in num_columns:
    sns.displot(data = df ,  x = col)

In [ ]:
cats=['ship_mode', 'segment', 'market', 'region',
       'category', 'sub_category', 'order_priority']
for col in cats:
    plt.figure(figsize=(15,10))
    df[col].value_counts().plot.pie(autopct='%.2f%%')    
    plt.title(f'Count of {col}')
    plt.show()

In [ ]:
cats

In [ ]:
sns.countplot(data = df , y = 'ship_mode')

In [ ]:
sns.countplot(data = df , y = 'segment')

In [ ]:
sns.countplot(data = df , y = 'market')

In [ ]:
sns.countplot(data = df , y = 'region')

In [ ]:
sns.countplot(data = df , y = 'category')

In [ ]:
sns.countplot(data = df , y = 'sub_category')

In [ ]:
sns.countplot(data = df , y = 'order_priority')

### Clean the data by fixing the errors using apply function

In [ ]:
df['market'].unique().tolist()

In [ ]:
df['state'].unique().tolist()

### Detect incomprehensible values 

In [ ]:
s='Al Iskandariyah'
def mod_state(state):
    if state == 'Al 11Iskandariyah':
        return s
    else:
        return state

In [ ]:
df['state']=df['state'].apply(mod_state)

In [ ]:
df['state']

In [ ]:
def mod_market (market):
    if market in ['?','??','xxx'] :
        return np.nan
    else:
        return market

In [ ]:
df['market']=df['market'].apply(mod_market)

In [ ]:
df['market'].unique()

### Here they have cleaned up the errors at the market & state column by changing the unknown symbols using [Apply Function]
## --------------------------------------------------------------------------------

### Check for missing values to  take it into consideration during analysis  

In [ ]:
df.isnull().mean()*100

### Drop Missing Values

In [ ]:
df.dropna(axis=0,inplace=True)

### Reset Index

In [ ]:
df.reset_index(drop=True,inplace=True)

### Apply Feature Engineering

In [ ]:
df['year'] = df['order_date'].dt.year
df['month'] = df['order_date'].dt.month_name(locale='English')
df['day'] = df['order_date'].dt.day

In [ ]:
df

### Extract Year and Month_name of order data
## -------------------------------------------------------------------------------------------------------------------------

### What is the best month for sale ?

In [ ]:
df.groupby('month')['sales'].sum().sort_values(ascending = False)

In [ ]:
sns.barplot(data=df, y='month', x='sales', estimator=sum, order=pd.value_counts(df['month']).index)

#### Here based on the gragh that [August and November] are the higher month in sale
## -----------------------------------------------------------------------------

### what is is the amount of Discount every month each year ?

In [ ]:
c = pd.pivot_table(data = df , index = 'year' , columns ='month' , values = 'discount' , aggfunc=sum)
sns.heatmap(c)

### What is the effect of Discount on Sales ?

In [ ]:
sns.lineplot(data=df, x='discount', y='profit',  hue='year', estimator=sum )

### What is the total sells for each region each year ?

In [ ]:
sns.barplot(data = df , y = 'region' , x = 'sales' , hue='year' , estimator=sum )

### What is the most discounted category ?

In [ ]:
df.groupby('category').sum()['discount'].sort_values(ascending = False)

In [ ]:
sns.barplot(data = df , x = 'category' , y = 'discount' , estimator=sum )

### What is the top 20 profited country ?

In [ ]:
df.groupby('country').sum()['profit'].sort_values(ascending = False).head(20)

In [ ]:
top20_country__profit = df.groupby('country').sum()['profit'].sort_values(ascending = False).head(20).index
top20_country_profit = df[df['country'].isin(top20_country__profit)]

In [ ]:
sns.barplot(data = top20_country_profit  , y = 'country' , x = 'profit' , estimator=sum , order=top20_country__profit)

### What is the top 20 profited state ?

In [ ]:
top20_state__profit = df.groupby('state').sum()['profit'].sort_values(ascending = False).head(20).index
top20_state_profit = df[df['state'].isin(top20_state__profit)]
top20_state_profit


In [ ]:
sns.barplot(data = top20_state_profit , y = 'state' , x = 'profit' , estimator=sum , order=top20_state__profit)

### What is the total profit for each category in each region ?

In [ ]:
b = pd.pivot_table(data = df , columns= 'region' , index ='category' , values ='profit' , aggfunc=sum)
sns.heatmap(b)

### what is is the amount of discount every month ?

In [ ]:
sns.barplot(data=df, y='month', x='discount', estimator=sum, order=pd.value_counts(df['month']).index)

### What is the most type of cutomer and with which segment dealing with our store ?

In [ ]:
sns.countplot(data=df, x='segment', hue='ship_mode')

### Which are the top 10 producted by sales & quantity & discount & profit ?

In [ ]:
pr_sales=pd.DataFrame(df.groupby('product_name').sum(['sales']))
pr_sales.sort_values(by=['sales'],inplace=True,ascending=False)
pr_sales

In [ ]:
pr_sales.head(10)